In [1]:
# Initial setup

# Import of relevant packages
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
from sklearn import metrics
from timeit import default_timer as timer
from sklearn.naive_bayes import CategoricalNB


from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.metrics import roc_curve, confusion_matrix, accuracy_score, recall_score, precision_score

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
# Set random seed 
RSEED = 42
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv('data/Inspiration_task 1.csv',)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x87 in position 10: invalid start byte